In [21]:
# Pake ini yang alternatif dataset juga

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
# import matplotlib.pyplot as plt
# import numpy as np
# import IPython.display as ipd
import noisereduce as nr
# from scipy.io import wavfile
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
# from scipy.spatial.distance import euclidean
# from sklearn.preprocessing import normalize
import numpy as np
import os

# Function to apply bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y_filtered = lfilter(b, a, data)
    return y_filtered


def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    # y = librosa.util.normalize(y)
    # y = y/np.max(np.abs(y))
    y = librosa.effects.preemphasis(y)
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    y = y/np.max(np.abs(y))
    # y = bandpass_filter(y, lowcut=100.0, highcut=3000.0, fs=sr, order=5)
    # y = shift_pitch_audio(y, sr)

    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    # y = nr.reduce_noise(y, sr, prop_decrease=0.5)
    # y = y/np.max(np.abs(y))
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True, # false = slaney, true = htk
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    # mfccs=mfccs[1:]
    # return librosa.util.normalize(mfccs.T)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(folder1, folder2, output_file):
    files1 = os.listdir(folder1)
    files2 = os.listdir(folder2)
    dtw_distances = []  # List untuk menyimpan hasil perbandingan jarak DTW
    error_point = 0
    with open(output_file, 'w') as f:
        for file1 in files1:
            for file2 in files2:
                audio_path1 = os.path.join(folder1, file1)
                audio_path2 = os.path.join(folder2, file2)

                mfccs1 = extraction(audio_path1)
                mfccs2 = extraction(audio_path2)

                dtw_distance = calculate_dtw(mfccs1, mfccs2)
                if (dtw_distance == 0):
                    result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                    print(result_line)
                    f.write(result_line)
                elif (dtw_distance < 0):
                    print(" ada yang kurang < 0")
                    error_point = 1
                    break;
                else:
                    dtw_distances.append(dtw_distance)  # Menambahkan jarak DTW ke dalam list
                    result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                    print(result_line)
                    f.write(result_line)
            if (error_point == 1):
                break;
        if error_point == 1:
            print("Perulangan dihentikan karena ada jarak DTW yang kurang dari 0")
            return

        # Menghitung rata-rata dari hasil perbandingan jarak DTW
        avg_dtw_distance = np.mean(dtw_distances)
        f.write(f"Rata-rata jarak DTW{folder1} dan {folder2}: {avg_dtw_distance}\n")
        print(f"Rata-rata jarak DTW{folder1} dan {folder2}: {avg_dtw_distance}")

folder1 = "../Dataset/01.Ha'/"
folder2 = "../Dataset/01.Ha'/"
output_file = '../new_hasil_banding/tes_prepro.txt'

main(folder1, folder2, output_file)

Jarak DTW antara Ha'_F1_11.wav dan Qaf_F1_11.wav: 21.02886357656374

Jarak DTW antara Ha'_F1_11.wav dan Qaf_F1_12.wav: 19.031684165727818

Jarak DTW antara Ha'_F1_11.wav dan Qaf_F1_13.wav: 19.837995167332142

Jarak DTW antara Ha'_F1_11.wav dan Qaf_F2_11.wav: 29.135988170088407

Jarak DTW antara Ha'_F1_11.wav dan Qaf_F2_12.wav: 25.19668076161587

Jarak DTW antara Ha'_F1_11.wav dan Qaf_F2_13.wav: 26.45634054382351

Jarak DTW antara Ha'_F1_11.wav dan Qaf_M1_11.wav: 23.546156789693672

Jarak DTW antara Ha'_F1_11.wav dan Qaf_M1_12.wav: 24.9912411440251

Jarak DTW antara Ha'_F1_11.wav dan Qaf_M1_13.wav: 22.601782336636397

Jarak DTW antara Ha'_F1_11.wav dan Qaf_M2_11.wav: 23.01309021738702

Jarak DTW antara Ha'_F1_11.wav dan Qaf_M2_12.wav: 24.000817202055636

Jarak DTW antara Ha'_F1_11.wav dan Qaf_M2_13.wav: 22.59251237162037

Jarak DTW antara Ha'_F1_11.wav dan Qaf_M3_1.wav: 26.259233251349677

Jarak DTW antara Ha'_F1_11.wav dan Qaf_M3_2.wav: 25.945457831036936

Jarak DTW antara Ha'_F1_11.wa

In [1]:
# Pake ini yang alternatif dataset juga

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
# import matplotlib.pyplot as plt
# import numpy as np
# import IPython.display as ipd
import noisereduce as nr
# from scipy.io import wavfile
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
# from scipy.spatial.distance import euclidean
# from sklearn.preprocessing import normalize
import numpy as np
import os
from IPython.display import clear_output

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(folder1, folder2, output_file):
    files1 = os.listdir(folder1)
    files2 = os.listdir(folder2)
    dtw_distances = []  # List untuk menyimpan hasil perbandingan jarak DTW
    error_point = 0
    with open(output_file, 'w') as f:
        for file1 in files1:
            for file2 in files2:
                mfcc_csv1 = os.path.join(folder1, file1)
                mfcc_csv2 = os.path.join(folder2, file2)

                mfccs1 = np.loadtxt(mfcc_csv1, delimiter=',')
                mfccs2 = np.loadtxt(mfcc_csv2, delimiter=',')

                # mfccs1 = mfccs1.T
                # mfccs1 = mfccs1[1:].T

                # mfccs2 = mfccs2.T
                # mfccs2 = mfccs2[1:].T
                

                dtw_distance = calculate_dtw(mfccs1, mfccs2)
                if (dtw_distance == 0):
                    result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                    print(result_line)
                    f.write(result_line)
                elif (dtw_distance < 0):
                    print(" ada yang kurang < 0")
                    error_point = 1
                    break;
                else:
                    dtw_distances.append(dtw_distance)  # Menambahkan jarak DTW ke dalam list
                    result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                    print(result_line)
                    f.write(result_line)
            if (error_point == 1):
                break;
        if error_point == 1:
            print("Perulangan dihentikan karena ada jarak DTW yang kurang dari 0")
            return

        # Menghitung rata-rata dari hasil perbandingan jarak DTW
        avg_dtw_distance = np.mean(dtw_distances)
        f.write(f"Rata-rata jarak DTW{folder1} dan {folder2}: {avg_dtw_distance}\n")
        print(f"Rata-rata jarak DTW{folder1} dan {folder2}: {avg_dtw_distance}")



root = "../Dataset_MFCC_noprem/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]

for folder1 in list_template_folders:
    for folder2 in list_template_folders:
        template_folder1 = os.path.join(root, folder1)
        template_folder2 = os.path.join(root, folder2)

        output_file = f"../new_hasil_banding/noprem_tesfix_FM_{folder1.strip('/')}_vs_{folder2.strip('/')}.txt"

        main(template_folder1, template_folder2, output_file)
        clear_output()

In [2]:
# Pake ini yang alternatif dataset juga

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
# import matplotlib.pyplot as plt
# import numpy as np
# import IPython.display as ipd
import noisereduce as nr
# from scipy.io import wavfile
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
# from scipy.spatial.distance import euclidean
# from sklearn.preprocessing import normalize
import numpy as np
import os
from IPython.display import clear_output

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(folder1, folder2, output_file):
    files1 = os.listdir(folder1)
    files2 = os.listdir(folder2)
    dtw_distances = []  # List untuk menyimpan hasil perbandingan jarak DTW
    error_point = 0
    with open(output_file, 'w') as f:
        for file1 in files1:
            for file2 in files2:
                mfcc_csv1 = os.path.join(folder1, file1)
                mfcc_csv2 = os.path.join(folder2, file2)

                mfccs1 = np.loadtxt(mfcc_csv1, delimiter=',')
                mfccs2 = np.loadtxt(mfcc_csv2, delimiter=',')

                # mfccs1 = mfccs1.T
                # mfccs1 = mfccs1[1:].T

                # mfccs2 = mfccs2.T
                # mfccs2 = mfccs2[1:].T
                

                dtw_distance = calculate_dtw(mfccs1, mfccs2)
                if (dtw_distance == 0):
                    result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                    print(result_line)
                    f.write(result_line)
                elif (dtw_distance < 0):
                    print(" ada yang kurang < 0")
                    error_point = 1
                    break;
                else:
                    dtw_distances.append(dtw_distance)  # Menambahkan jarak DTW ke dalam list
                    result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                    print(result_line)
                    f.write(result_line)
            if (error_point == 1):
                break;
        if error_point == 1:
            print("Perulangan dihentikan karena ada jarak DTW yang kurang dari 0")
            return

        # Menghitung rata-rata dari hasil perbandingan jarak DTW
        avg_dtw_distance = np.mean(dtw_distances)
        f.write(f"Rata-rata jarak DTW{folder1} dan {folder2}: {avg_dtw_distance}\n")
        print(f"Rata-rata jarak DTW{folder1} dan {folder2}: {avg_dtw_distance}")



root = "../Dataset_MFCC_male_only/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]

for folder1 in list_template_folders:
    for folder2 in list_template_folders:
        template_folder1 = os.path.join(root, folder1)
        template_folder2 = os.path.join(root, folder2)

        output_file = f"../new_hasil_banding/male_only_tesfix{folder1.strip('/')}_vs_{folder2.strip('/')}.txt"

        main(template_folder1, template_folder2, output_file)
        clear_output()

In [1]:
# Pake ini yang alternatif dataset juga / banding yang folder sama saja

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
# import matplotlib.pyplot as plt
# import numpy as np
# import IPython.display as ipd
import noisereduce as nr
# from scipy.io import wavfile
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
# from scipy.spatial.distance import euclidean
# from sklearn.preprocessing import normalize
import numpy as np
import os
from IPython.display import clear_output

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(folder1, folder2, output_file):
    files1 = os.listdir(folder1)
    files2 = os.listdir(folder2)
    dtw_distances = []  # List untuk menyimpan hasil perbandingan jarak DTW
    error_point = 0
    with open(output_file, 'w') as f:
        for file1 in files1:
            for file2 in files2:
                mfcc_csv1 = os.path.join(folder1, file1)
                mfcc_csv2 = os.path.join(folder2, file2)

                mfccs1 = np.loadtxt(mfcc_csv1, delimiter=',')
                mfccs2 = np.loadtxt(mfcc_csv2, delimiter=',')

                # mfccs1 = mfccs1.T
                # mfccs1 = mfccs1[1:].T

                # mfccs2 = mfccs2.T
                # mfccs2 = mfccs2[1:].T
                

                dtw_distance = calculate_dtw(mfccs1, mfccs2)
                if (dtw_distance == 0):
                    result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                    print(result_line)
                    f.write(result_line)
                elif (dtw_distance < 0):
                    print(" ada yang kurang < 0")
                    error_point = 1
                    break;
                else:
                    dtw_distances.append(dtw_distance)  # Menambahkan jarak DTW ke dalam list
                    result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                    print(result_line)
                    f.write(result_line)
            if (error_point == 1):
                break;
        if error_point == 1:
            print("Perulangan dihentikan karena ada jarak DTW yang kurang dari 0")
            return

        # Menghitung rata-rata dari hasil perbandingan jarak DTW
        avg_dtw_distance = np.mean(dtw_distances)
        f.write(f"Rata-rata jarak DTW{folder1} dan {folder2}: {avg_dtw_distance}\n")
        print(f"Rata-rata jarak DTW{folder1} dan {folder2}: {avg_dtw_distance}")



root = "../Dataset_Alt_MFCC_male_only/"

list_template_folders = [
    # ("01.Ha'/"),
    # ("02.Kha'/"),
    # ("03.Shad/"),
    # ("04.Dhad/"),
    # ("05.Tha'/"),
    # ("06.Dhza'/"),
    # ("07.'AIn/"),
    # ("08.Ghain/"),
    # ("09.Qaf/"),
    # ("10.Ha^'/"),
    ("01.Ha'_Alt/"),
    ("02.Kha'_Alt/"),
    ("03.Shad_Alt/"),
    ("04.Dhad_Alt/"),
    ("05.Tha'_Alt/"),
    ("06.Dhza'_Alt/"),
    ("07.'AIn_Alt/"),
    ("08.Ghain_Alt/"),
    ("09.Qaf_Alt/"),
    ("10.Ha^'_Alt/")
]

for folder1 in list_template_folders:
    folder2 = folder1
    template_folder1 = os.path.join(root, folder1)
    template_folder2 = os.path.join(root, folder2)

    output_file = f"../new_hasil_banding/Alt_male_only_tesfix{folder1.strip('/')}_vs_{folder2.strip('/')}.txt"

    main(template_folder1, template_folder2, output_file)
    clear_output()

In [16]:
# banding sample_dataset

import librosa
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import noisereduce as nr
from scipy.io import wavfile
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import normalize
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    # y = librosa.util.normalize(y)
    # y = librosa.effects.preemphasis(y)
    y = nr.reduce_noise(y, sr)
    # y = normalize(y[:, np.newaxis], axis=0).ravel()
    # y = librosa.effects.preemphasis(y)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, path = fastdtw(mfccs1, mfccs2, dist=cosine)
    # distance = distance / len(path)
    return distance

def main(folder1, folder2, output_file):
    files1 = os.listdir(folder1)
    files2 = os.listdir(folder2)

    with open(output_file, 'w') as f:
        for file1 in files1:
            for file2 in files2:
                audio_path1 = os.path.join(folder1, file1)
                audio_path2 = os.path.join(folder2, file2)

                mfccs1 = extraction(audio_path1)
                mfccs2 = extraction(audio_path2)

                dtw_distance = calculate_dtw(mfccs1, mfccs2)

                result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                print(result_line)
                f.write(result_line)



folder1 = "../sample_dataset/ha"
folder2 = "../sample_dataset/ha"
output_file = '../hasil_perbandingan/441,fft1024,512hop_ha-ha.txt'

main(folder1, folder2, output_file)

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_1_1.wav: 0

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_1_2.wav: 0.12703062309865432

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_1_3.wav: 0.11665571348832193

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_1_4.wav: 0.1751494297681293

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_2_1.wav: 0.7798749055681887

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_2_2.wav: 0.7465919337169906

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_2_3.wav: 0.5959441225697084

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_2_4.wav: 0.5942621163095457

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_4_1.wav: 0.25282896534140564

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_4_2.wav: 0.2843388136447734

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_4_3.wav: 0.3551644991619759

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_4_4.wav: 0.36202343612127585

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_5_1.wav: 0.3743276951023323

Jarak DTW antara 27_ha_1_1.wav dan 27_ha_5_2.wav: 0.6336261846534614

Jarak DTW antara 27_ha_1_1.wav da

In [37]:
from python_speech_features import mfcc
# from python_speech_features import delta
from fastdtw import fastdtw
# import scipy.io.wavfile as wav
import numpy as np
from scipy.spatial.distance import cosine
import librosa
import noisereduce as nr
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=16000)
    y, _ = librosa.effects.trim(y, top_db=20)
    # y = librosa.effects.preemphasis(y)
    y = nr.reduce_noise(y, sr)
    y = normalize(y[:, np.newaxis], axis=0).ravel()
    return y,sr

def extraction(audio):
    sig, rate = preprocessing(audio)
    mfcc_feat = mfcc(sig,
                samplerate=16000,
                winlen=0.025,
                winstep=0.01,
                numcep=12,
                nfilt=20,
                preemph=0.97,
                ceplifter=22,
                winfunc=np.hamming)
    # mfcc_feat = delta(mfcc_feat, 1)
    # mfcc_feat = delta(mfcc_feat, 2)
    
    mfcc_feat = librosa.util.normalize(mfcc_feat)
    # return np.mean(mfcc_feat, axis=0)
    return mfcc_feat



def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(folder1, folder2, output_file):
    files1 = os.listdir(folder1)
    files2 = os.listdir(folder2)

    with open(output_file, 'w') as f:
        for file1 in files1:
            for file2 in files2:
                audio_path1 = os.path.join(folder1, file1)
                audio_path2 = os.path.join(folder2, file2)

                mfccs1 = extraction(audio_path1)
                mfccs2 = extraction(audio_path2)

                dtw_distance = calculate_dtw(mfccs1, mfccs2)

                result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                print(result_line)
                f.write(result_line)



folder1 = "../sample_dataset/qa"
folder2 = "../sample_dataset/qa"
output_file = '../hasil_perbandingan/ps_sample_qa-qa.txt'

main(folder1, folder2, output_file)

Jarak DTW antara 21_qa_2_1.wav dan 21_qa_2_1.wav: 0

Jarak DTW antara 21_qa_2_1.wav dan 21_qa_2_2.wav: 3.474618118130489

Jarak DTW antara 21_qa_2_1.wav dan 21_qa_2_3.wav: 4.3047528342247245

Jarak DTW antara 21_qa_2_1.wav dan 21_qa_2_4.wav: 11.13968913063045

Jarak DTW antara 21_qa_2_1.wav dan 21_qo_1_1.wav: 21.776375213520765

Jarak DTW antara 21_qa_2_1.wav dan 21_qo_1_2.wav: 16.400970884294633

Jarak DTW antara 21_qa_2_1.wav dan 21_qo_1_3.wav: 18.96741399132261

Jarak DTW antara 21_qa_2_1.wav dan 21_qo_1_4.wav: 24.98459972565743

Jarak DTW antara 21_qa_2_1.wav dan 21_qo_4_1.wav: 20.027541144506667

Jarak DTW antara 21_qa_2_1.wav dan 21_qo_4_2.wav: 22.374729840762832

Jarak DTW antara 21_qa_2_1.wav dan 21_qo_4_3.wav: 23.56912741997396

Jarak DTW antara 21_qa_2_1.wav dan 21_qo_4_4.wav: 23.56522976742031

Jarak DTW antara 21_qa_2_1.wav dan 7_qho_5_1.wav: 20.362315225369247

Jarak DTW antara 21_qa_2_1.wav dan 7_qho_5_2.wav: 21.006938053105852

Jarak DTW antara 21_qa_2_1.wav dan 7_qho_5_

In [36]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import noisereduce as nr
from scipy.io import wavfile
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import normalize
import os
from dtw import *

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=16000)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = librosa.util.normalize(y)
    # y = librosa.effects.preemphasis(y)
    y = nr.reduce_noise(y, sr)
    y = normalize(y[:, np.newaxis], axis=0).ravel()
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 htk=False,
                                 hop_length=256,
                                 n_fft=512,
                                 norm='ortho',
                                 n_mels=20)
    # mfccs = librosa.util.normalize(mfccs)
    return mfccs


def calculate_dtw(mfccs1, mfccs2):
    alignment = dtw(mfccs1, mfccs2, dist=cosine)
    return alignment

def main(folder1, folder2, output_file):
    files1 = os.listdir(folder1)
    files2 = os.listdir(folder2)

    with open(output_file, 'w') as f:
        for file1 in files1:
            for file2 in files2:
                audio_path1 = os.path.join(folder1, file1)
                audio_path2 = os.path.join(folder2, file2)

                mfccs1 = extraction(audio_path1)
                mfccs2 = extraction(audio_path2)

                dtw_distance = calculate_dtw(mfccs1, mfccs2)

                result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                print(result_line)
                f.write(result_line)



folder1 = "../Dataset/01.Ha\'/"
folder2 = "../Dataset/01.Ha\'/"
output_file = '../hasil_perbandingan/dtw-librosa_perbandingan1.txt'

main(folder1, folder2, output_file)

TypeError: 'str' object is not callable

In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import noisereduce as nr
from scipy.io import wavfile
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import normalize
import os


def preprocessing(audio):
    y , sr = librosa.load(audio, sr=16000)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = librosa.util.normalize(y)
    # y = librosa.effects.preemphasis(y)
    y = nr.reduce_noise(y, sr)
    y = normalize(y[:, np.newaxis], axis=0).ravel()
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 htk=False,
                                 hop_length=256,
                                 n_fft=512,
                                 norm='ortho',
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T



audio_male = "../Dataset/01.Ha\'/Ha\'_M1_1.wav"
audio_female = "../Dataset/01.Ha\'/Ha\'_F1_1.wav"

mfcc_male = extraction(audio_male)
mfcc_female = extraction(audio_female)

fig = plt.figure(figsize=(14,7))
fig.patch.set_facecolor('white')
plt.plot
